In [6]:
import pandas as pd
import bsa_utils
import utils

In [7]:
bsa_utils.show_available_datasets()

23362
25521
consolidated-pharmaceutical-list
contractor-details
dental-activity-delivered-by-newly-qualified-foundation-dentists
dispensing-practice-name-and-address
english-contractor-monthly-general-dental-activity
english-contractor-monthly-general-dental-and-orthodontic-contractual-dataset
english-contractor-monthly-orthodontic-activity
english-prescribing-data-epd
english-prescribing-dataset-epd-with-snomed-code
hormone-replacement-therapy-prescription-prepayment-certificate-hrt-ppc-volumes
hospital-prescribing-dispensed-in-the-community
national-uda-by-performer-for-financial-years-2016-2024
opendata-nhsbsa-net-dataset-foi-25455
open-data-portal-usage
pharmacy-contract-fp57-refunds
pharmacy-openings-and-closures
pharmacy-quality-scheme-data-2022-23
prescriber-details
prescription-cost-analysis-pca-annual-statistics
prescription-cost-analysis-pca-monthly-data
regional-uda-by-performer
secondary-care-medicines-data
secondary-care-medicines-data-indicative-price


In [21]:
#FIND NEW PRODUCTS
dataset_id = "english-prescribing-data-epd" # Dateset ID
date_from = "201801" # Can be "YYYYMM" or "earliest" or "latest", default="earliest"
date_to = "latest-1" # Can be "YYYYMM" or "latest" or "latest-1", default="latest"
sql = (
    "SELECT DISTINCT BNF_CODE, BNF_DESCRIPTION, CHEMICAL_SUBSTANCE_BNF_DESCR "
    "{FROM_TABLE}"
)

# Fetch data using BSA API
existing_data_extract=bsa_utils.FetchData(resource=dataset_id, date_from=date_from, date_to=date_to, sql=sql, cache=True)

# Save data to file
#df.to_csv("../data/data.csv", index=False)

date_from = "latest" # Can be "YYYYMM" or "earliest" or "latest", default="earliest"
date_to = "" # Can be "YYYYMM" or "latest" or "latest-1", default="latest"
sql = (
    "SELECT DISTINCT BNF_CODE, BNF_DESCRIPTION, CHEMICAL_SUBSTANCE_BNF_DESCR "
    "{FROM_TABLE}"
)

# Fetch data using BSA API
latest_data_extract=bsa_utils.FetchData(resource=dataset_id, date_from=date_from, date_to=date_to, sql=sql)

# Compare 2 dataframe - return rows with items which are only in new dataframe.
#new_items=utils.show_new_items(existing_data_extract.results(), latest_data_extract.results())

# Temporarily set the display option to show all rows for one print statement
#with pd.option_context('display.max_rows', None, 'display.max_colwidth', 100):
    #display(new_items)

In [26]:
def show_new_items(df_existing, df_latest):
    df_existing
    #Remove duplicates from df_existing
    #df_existing = df_existing.drop_duplicates(subset=['BNF_CODE', 'BNF_DESCRIPTION'])

    #Remove duplicates from df_latest 

    # Merge the dataframes with an indicator
    merged_df = df_latest.merge(df_existing, on=['BNF_CODE'], how='inner', suffixes=('_latest', '_existing'))

    print (merged_df.head(10))

    # Filter rows where BNF_DESCRIPTIONs do not match
    #updated_codes = merged_df[merged_df['BNF_DESCRIPTION_latest'] != merged_df['BNF_DESCRIPTION_existing']]

    # Select the relevant columns and rename them
    #updated_codes = updated_codes[['BNF_CODE', 'BNF_DESCRIPTION_latest']]
    #updated_codes = updated_codes.rename(columns={'BNF_DESCRIPTION_latest': 'BNF_DESCRIPTION'})

    # Sort the dataframe by BNF_CODE
    #updated_codes = sort_by_bnf_code(updated_codes)

    #return updated_codes

def find_bnf_code_only_in_latest(df_existing, df_latest):
    # Find BNF_CODEs that are only in df_latest
    latest_codes = set(df_latest['BNF_CODE'])
    existing_codes = set(df_existing['BNF_CODE'])
    unique_codes = latest_codes - existing_codes
    
    # Filter df_latest based on unique BNF_CODEs
    result = df_latest[df_latest['BNF_CODE'].isin(unique_codes)]
    result=sort_by_bnf_code(result)
    return result

def find_bnf_description_only_in_latest(df_existing, df_latest):
    # Find BNF_DESCRIPTIONs that are only in df_latest
    latest_descriptions = set(df_latest['BNF_DESCRIPTION'])
    existing_descriptions = set(df_existing['BNF_DESCRIPTION'])
    unique_descriptions = latest_descriptions - existing_descriptions
    
    # Filter df_latest based on unique BNF_DESCRIPTIONs
    result = df_latest[df_latest['BNF_DESCRIPTION'].isin(unique_descriptions)]
    result=sort_by_bnf_code(result)
    return result

def find_chemical_substance_bnf_descr_only_in_latest(df_existing, df_latest):
    # Find CHEMICAL_SUBSTANCE_BNF_DESCRs that are only in df_latest
    latest_substances = set(df_latest['CHEMICAL_SUBSTANCE_BNF_DESCR'])
    existing_substances = set(df_existing['CHEMICAL_SUBSTANCE_BNF_DESCR'])
    unique_substances = latest_substances - existing_substances
    
    # Filter df_latest based on unique CHEMICAL_SUBSTANCE_BNF_DESCRs
    result = df_latest[df_latest['CHEMICAL_SUBSTANCE_BNF_DESCR'].isin(unique_substances)]
    result=sort_by_bnf_code(result)
    return result

def sort_by_bnf_code(df):
    # Make a copy of the dataframe to avoid SettingWithCopyWarning
    df = df.copy()

    # Add column BNF_CHAPTER which is the first 2 characters of BNF_CODE
    df.loc[:, 'BNF_CHAPTER'] = df['BNF_CODE'].str[:2]
    # Add column BNF_SECTION which is the 3rd and 4th characters of BNF_CODE
    df.loc[:, 'BNF_SECTION'] = df['BNF_CODE'].str[2:4]
    # Add column BNF_PARAGRAPH which is the 5th and 6th characters of BNF_CODE
    df.loc[:, 'BNF_PARAGRAPH'] = df['BNF_CODE'].str[4:6]
    # Add column BNF_SUBPARAGRAPH which is the 7th character of BNF_CODE
    df.loc[:, 'BNF_SUBPARAGRAPH'] = df['BNF_CODE'].str[6]

    # Sort the dataframe by BNF_CHAPTER Ascending, BNF_SECTION Ascending, BNF_PARAGRAPH Ascending, BNF_SUBPARAGRAPH Ascending
    df = df.sort_values(by=['BNF_CHAPTER', 'BNF_SECTION', 'BNF_PARAGRAPH', 'BNF_SUBPARAGRAPH'])

    # Drop the intermediate columns
    df = df.drop(columns=['BNF_CHAPTER', 'BNF_SECTION', 'BNF_PARAGRAPH', 'BNF_SUBPARAGRAPH'])

    # Reset the index
    df = df.reset_index(drop=True)

    return df

In [29]:
result=find_chemical_substance_bnf_descr_only_in_latest(existing_data_extract.results(), latest_data_extract.results())
# Temporarily set the display option to show all rows for one print statement
with pd.option_context('display.max_rows', None, 'display.max_colwidth', 100):
    display(result)

,BNF_CODE,BNF_DESCRIPTION,CHEMICAL_SUBSTANCE_BNF_DESCR
0,0309020ABBBAAAA,Cough Nurse Night Time Liquid,Diphenhydramine hydrochloride/pholcodine
1,0407042U0AAAAAA,Atogepant 10mg tablets,Atogepant
2,0407042U0AAABAB,Atogepant 60mg tablets,Atogepant
3,0407042U0BBAAAA,Aquipta 10mg tablets,Atogepant


In [30]:
result=find_bnf_code_only_in_latest(existing_data_extract.results(), latest_data_extract.results())
# Temporarily set the display option to show all rows for one print statement
with pd.option_context('display.max_rows', None, 'display.max_colwidth', 100):
    display(result)

,BNF_CODE,BNF_DESCRIPTION,CHEMICAL_SUBSTANCE_BNF_DESCR
0,0107010AAAAAQAQ,Diltiazem 0.4% ointment,Diltiazem hydrochloride
1,0205010Y0BBACAC,Revatio 10mg/12.5ml solution for injection vials,Sildenafil(Vasodilator Antihypertensive)
2,0205052AEAAADAD,Sacubitril 6mg / Valsartan 6mg gran in caps for opening,Sacubitril/valsartan
3,0206010K0CVAAAE,Medomon XL 60mg tablets,Isosorbide mononitrate
4,0208020X0BBAIAI,Pradaxa 150mg granules sachets,Dabigatran etexilate
5,0302000C0AAAKAK,Beclometasone 200microgram inhalation powder blisters,Beclometasone dipropionate
6,0304020AAAAAAAA,Benralizumab 30mg/1ml inj pre-filled syringes,Benralizumab
7,0309020ABBBAAAA,Cough Nurse Night Time Liquid,Diphenhydramine hydrochloride/pholcodine
8,0402010J0AABGBG,Haloperidol 200micrograms/5ml oral liquid,Haloperidol
9,0402020AAAAAEAE,Risperidone 100mg prolonged-release inj pre-filled syringes,Risperidone


In [31]:
result=find_bnf_description_only_in_latest(existing_data_extract.results(), latest_data_extract.results())
# Temporarily set the display option to show all rows for one print statement
with pd.option_context('display.max_rows', None, 'display.max_colwidth', 100):
    display(result)

,BNF_CODE,BNF_DESCRIPTION,CHEMICAL_SUBSTANCE_BNF_DESCR
0,0107010AAAAAQAQ,Diltiazem 0.4% ointment,Diltiazem hydrochloride
1,0205010Y0BBACAC,Revatio 10mg/12.5ml solution for injection vials,Sildenafil(Vasodilator Antihypertensive)
2,0205052AEAAADAD,Sacubitril 6mg / Valsartan 6mg gran in caps for opening,Sacubitril/valsartan
3,0206010K0CVAAAE,Medomon XL 60mg tablets,Isosorbide mononitrate
4,0208020X0BBAIAI,Pradaxa 150mg granules sachets,Dabigatran etexilate
5,0302000C0AAAKAK,Beclometasone 200microgram inhalation powder blisters,Beclometasone dipropionate
6,0304020AAAAAAAA,Benralizumab 30mg/1ml inj pre-filled syringes,Benralizumab
7,0309020ABBBAAAA,Cough Nurse Night Time Liquid,Diphenhydramine hydrochloride/pholcodine
8,0402010J0AABGBG,Haloperidol 200micrograms/5ml oral liquid,Haloperidol
9,0402020AAAAAEAE,Risperidone 100mg prolonged-release inj pre-filled syringes,Risperidone
